Определение языка (language detection)
--------------------

* **Множество случаев** — тексты на разных языках
* **Множество классов** — языки

In [ ]:
Абсолютно все свои заметки я написала здесь. Надеюсь, это будет считаться в качестве отчёта.

Я решила взять 4 языка, не очень связанных друг с другом. Это монгольский, польский, русинский и немецкий. Никакой особой логики в выборе языков нет, просто было интересно.
Первым делом выкачиваем статьи.

In [1]:
langues = ['mn', 'pl', 'rue', 'de']

In [2]:
def get_texts_for_lang(lang, n=10): # функция для скачивания статей из википедии
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

Изначально я делала на том же составе языков, что был дан в примере. И по какой-то причине он работает гораздо быстрее, чем мой. В первый раз на выкачку статей ушло пол часа, потом стандартно минут 20.

In [3]:
import wikipedia # скачиваем по 100 статей для каждого языка. Это может занять какое-то время (5-10 минут. как правило)

wiki_texts = {}
for lang in langues:
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts[lang]))

Skipping page 2011-2012 оны НБА-гийн улирал
mn 99
Skipping page Beveren (stacja kolejowa)


C:\Users\1\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\1\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Skipping page Złoty Krążek (Słowacja)
Skipping page Coppa Italia
Skipping page Kaplica papieska
Skipping page Blackout
pl 95
rue 100
Skipping page Gipshütte
Skipping page Karl Findeisen
Skipping page Bundesgericht
Skipping page Großer Vogelsand
Skipping page Tanck
Skipping page Erich Zieger
Skipping page Gaius Claudius Pulcher
Skipping page Richard Portman
Skipping page Krieger
de 91


Проверяем, скачались ли тексты.

In [4]:
print(wiki_texts['rue'][0])
print(wiki_texts['de'][0])

922

 Подїї 


 Народили ся 


 Вмерли 
Spitalerstraße
Die Spitalerstraße in Hamburg ist eine der zentralen Einkaufsstraßen der Stadt. Sie verbindet den mittleren Teil der Mönckebergstraße und den Gerhart-Hauptmann-Platz auf Höhe des U-Bahnhof Mönckebergstraße mit den östlich gelegenen, ineinander übergehenden Straßen Glockengießerwall und Steintorwall gegenüber dem westlichen Eingang zur Wandelhalle des Hauptbahnhofs.
Nach Passantenzählungen in Einkaufsstraßen war die Spitalerstraße 2013 mit 9.215 Fußgängern in einer Stunde, nach der Mönckebergstraße, die zweitstärkst frequentierte Straße in Hamburg. Im bundesweiten Vergleich rangierte die Spitalerstraße im Jahr 2013 auf Platz 12 und lag im Jahr 2011 sogar mit 11.190 Fußgängern pro Stunde auf Platz 7 nur knapp hinter der Mönckebergstraße.
Außer den Blickachsen auf die Halle des Hauptbahnhofs im Osten und die am Rande der Achse stehende Petrikirche im Westen wird die Spitalerstraße wesentlich durch Geschäftsgebäude geprägt, die überwie

### Первый метод: частотные слова

In [5]:
import collections
import codecs
import sys
import re

In [6]:
from string import punctuation, digits
punctuation = set(punctuation + '«»—…“”\n\t' + digits)

С очисткой у меня возникли небольшие проблемы. У знаков отсутствовала какая-либо закономерность, поэтому многие из них пришлось удалять отдельно, чтобы уж наверняка. 
Очистку вынесла в функцию, так как ниже использовала её отдельно от токенизации.

In [7]:
def clean(text):
    text = re.sub(r'\.| - =|,|; !', ' ', text)
    text = re.sub(r'=|·', ' ', text)
    text = re.sub(r';', ' ', text)
    text = re.sub(r'\+|/', ' ', text)
    text = re.sub(r'’|\'', ' ', text)
    text = re.sub('(.*?)-|:(.*?)', ' ', text)
    text = re.sub(r'\d|  |\[|\]|\(|\)|\\|—|\"|\„|\n|&|»|«', '', text)
    return text

In [8]:
def tokenize(text):
    text = clean(text)
    return text.split(' ')

Функция для собирания частотных слов. Возвращает сет частотных слов для языка.

In [28]:
def get_freq(wiki_texts, lang):
    freqs = collections.defaultdict(lambda: 0)
    corpus = wiki_texts[lang]
    for article in corpus:
        for word in tokenize(article.replace('\n', '').lower()):
            freqs[word] += 1
    return set(freqs.keys())

In [29]:
freq_de = get_freq(wiki_texts, 'de')
freq_rue = get_freq(wiki_texts, 'rue')
freq_pl = get_freq(wiki_texts, 'pl')
freq_mn = get_freq(wiki_texts, 'mn')

Очищаем сеты от повторений и кладём их в словарь формата {'язык':'частотные слова'}. В немецком (и не только в нём) мне попадались иероглифы. Удалить их никак не смогла, так как не знаю точно, сколько их и какие. Но этот факт не повлиял на результаты, поэтому оставила так, как есть.

In [30]:
dict_freq = {}
dict_freq['de'] = freq_de - freq_rue - freq_pl - freq_mn
dict_freq['rue'] = freq_rue - freq_de - freq_pl - freq_mn
dict_freq['pl'] = freq_pl - freq_de - freq_rue - freq_mn
dict_freq['mn'] = freq_mn - freq_de - freq_rue - freq_pl

Функция по определению языка на основе частотных слов.

In [31]:
def predict_language_f(text, dict_freq, langues):
    text_chars = tokenize(text.lower())
    lang2sim = {}
    
    for lang in langues:
        intersect = len(set(text_chars) & dict_freq[lang])
        lang2sim[lang] = intersect
    
    return max(lang2sim, key=lambda x: lang2sim[x])

### Второй метод: частотные символьные n-граммы

Функция, которая преобразовывает строку в массив n-грамм заданной длины.

In [13]:
from itertools import islice, tee

def make_ngrams(text):
    N = 3 # задаем длину n-граммы
    text = clean(text)
    text = text.strip()
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    ngrams_fin = [''.join(x) for x in ngrams]
    return ngrams_fin

Функция для собирания частотных n-грамм. Возвращает сет n-грамм для языка.

In [14]:
def get_ngrams(wiki_texts, lang):
    ngrams_l= collections.defaultdict(lambda: 0)
    corpus = wiki_texts[lang]
    for article in corpus:
        for ngram in make_ngrams(article.replace('\n', '').lower()):
            ngrams_l[ngram] += 1
    return set(ngrams_l.keys())

In [27]:
ngrams_de = get_ngrams(wiki_texts, 'de')
ngrams_rue = get_ngrams(wiki_texts, 'rue')
ngrams_pl = get_ngrams(wiki_texts, 'pl')
ngrams_mn = get_ngrams(wiki_texts, 'mn')

Очищаем сеты от повторений и кладём в словарь формата {'язык':'n-граммы'}

In [20]:
dict_ngrams = {}
dict_ngrams['de'] = ngrams_de - ngrams_rue - ngrams_pl - ngrams_mn
dict_ngrams['rue'] = ngrams_rue - ngrams_de - ngrams_pl - ngrams_mn
dict_ngrams['pl'] = ngrams_pl - ngrams_de - ngrams_rue - ngrams_mn
dict_ngrams['mn'] = ngrams_mn - ngrams_de - ngrams_rue - ngrams_pl

Функция по определению языка на основе частотных слов.

In [21]:
def predict_language_n(text, dict_ngrams, langues):
    text_chars = make_ngrams(text.lower())
    lang2sim = {}
    
    for lang in langues:
        intersect = len(set(text_chars) & dict_ngrams[lang])
        lang2sim[lang] = intersect
    
    return max(lang2sim, key=lambda x: lang2sim[x])

### Определяем язык

In [125]:
def predict_lang(wiki_texts, langues, dict_freq, dict_ngrams):
    check_n = []
    check_f = []
    for lang in langues:
        for texts in wiki_texts[lang]:
            c_f = predict_language_f(texts, dict_freq, langues)
            check_f.append(c_f == lang)
            #check_f.append(lang)
            c_n = predict_language_n(texts, dict_ngrams, langues)
            check_n.append(c_n == lang)
            
    return check_f, check_n

In [126]:
pred_freq, pred_ngram = predict_lang(wiki_texts, langues, dict_freq, dict_ngrams)

In [83]:
False in pred_freq

True

In [127]:
for i in range (0, 770):
    if pred_freq[i] is False:
        print(pred_freq[i+1])

de


In [66]:
False in pred_ngram

False

Метод частотных слов выдаёт одну ошибку в предложении на немецком языке. Немецкий язык определяется как русинский.
Теперь сделаем проверку на другой выборке текстов.

In [41]:
wiki_texts_check = {}
for lang in langues:
    wiki_texts_check[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts_check[lang]))

mn 100


C:\Users\1\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\1\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Skipping page (140) Siwa
Skipping page Saint-Estèphe
Skipping page Zarzecze (obwód lwowski)
Skipping page Pleuroceras
Skipping page Marigot
pl 95
rue 100
Skipping page Mitsuru Satō
Skipping page ALJ
Skipping page Adolf Sonnenschein (Verwaltungsjurist)
Skipping page Wilhelm Schacht
Skipping page Häselbach
Skipping page Schiffskiel
Skipping page Ninowa
Skipping page Blesendorf
Skipping page Tayabas
Skipping page Yellowstone Township
de 90


In [119]:
def predict_lang(wiki_texts_check, langues, dict_freq, dict_ngrams):
    check_n = []
    check_f = []
    for lang in langues:
        for texts in wiki_texts_check[lang]:
            c_f = predict_language_f(texts, dict_freq, langues)
            check_f.append(c_f == lang)
            #check_f.append(lang)
            c_n = predict_language_n(texts, dict_ngrams, langues)
            check_n.append(c_n == lang)
            #check_n.append(lang)
            
    return check_f, check_n

In [120]:
pred_freq_check, pred_ngram_check = predict_lang(wiki_texts_check, langues, dict_freq, dict_ngrams)

In [95]:
False in pred_freq_check

True

In [123]:
for i in range (0, 770):
    if pred_freq_check[i] is False:
        print(pred_freq_check[i+1])

rue


In [101]:
len(pred_freq_check)

770

In [47]:
False in pred_ngram_check

True

In [118]:
for i in range(0, 770):
    if pred_ngram_check[i] is False:
        print(pred_ngram_check[i+1])

mn
de


Результат немного ухудшился. Однако нельзя сказать, что это зависит от какого-то конкретного языка, так как на тестовой выборке ошибки появляются в разных языках. Единственный язык, который всегда определяется правильно - польский.
В случае n-грамм монгольский и немецкий языки по одному разу определяются как польский. В случае частотных слов русинский определяется как монгольский. 

### Вывод

Я считаю, что оба метода работают одинаково хорошо. Ошибки в обоих случаях единичны и не систематичны. Эти языки не похожи, возможн, ошибка возникла из-за иноязычных вкраплений в статьях википедии.
В эту тетрадку не вошло, но до финальной записи я проверяла превоначальный набор языков на этих же функциях (фр, ук, бел, каз), и ошибок не было.